In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from hyperopt import hp, tpe, fmin, Trials
from tqdm import tqdm

import datetime
import copy
import json

import sys
sys.path.append('../../')

from data.dataloader import Covid19IndiaLoader
# from data.processing import get_data
from data.processing import granular, processing

from models.seir.seir_testing import SEIR_Testing
from models.seir.seirhd import SEIRHD
from models.seir.seir_movement import SEIR_Movement
from models.seir.seir_movement_testing import SEIR_Movement_Testing

from main.seir.fitting import single_fitting_cycle, get_variable_param_ranges
from main.seir.forecast import get_forecast, create_region_csv, create_all_csvs, write_csv
from utils.create_report import create_report
from utils.loss import Loss_Calculator
from viz import plot_forecast

In [ ]:
from utils.smooth_jump import *
from viz import plot_smoothing

## Assign Filename var

In [ ]:
filename = '../../data/data/mumbai/case_summary_2006.csv'

In [ ]:
strat_filename = '../../data/data/mumbai/case_summary_expanded_2006.csv'

In [ ]:
df_strat = granular.get_data(filename=strat_filename)

In [ ]:
df = processing.get_data(filename=filename, disable_tracker=True)

In [ ]:
df

In [ ]:
df_smoothed = smooth_big_jump(df)

In [ ]:
df_strat_smoothed = copy.copy(df_strat)

diff_array = df_smoothed.loc[df_smoothed['date'].isin(df_strat['date']), 'active'].reset_index(drop=True) - df_strat['active']

for column in df_strat_smoothed.loc[:, 'stable_asymptomatic':].columns:
    df_strat_smoothed[column] += diff_array*(df_strat_smoothed[column]/df_strat_smoothed['active'])

base_columns = ['total', 'active', 'recovered', 'deceased']
df_strat_smoothed.loc[:, base_columns] = df_smoothed.loc[df_smoothed['date'].isin(df_strat['date']), base_columns].reset_index(drop=True)

In [ ]:
all_compartments = ['total', 'active', 'recovered', 'deceased', 'stable_asymptomatic', 'stable_symptomatic', 'critical', 'hq', 'non_o2_beds', 'o2_beds', 'icu', 'ventilator']

In [ ]:
d1 = '2020-06-10'
length = (datetime.datetime.strptime(d1, '%Y-%m-%d') - df_strat_smoothed.loc[0, 'date']).days
df_strat_smoothed = smooth_big_jump_helper(
    df_strat_smoothed, 'stable_symptomatic', 'stable_asymptomatic', d1, smoothing_length=length, method='weighted')

d1 = '2020-06-15'
length = (datetime.datetime.strptime(d1, '%Y-%m-%d') - df_strat_smoothed.loc[0, 'date']).days
df_strat_smoothed = smooth_big_jump_helper(
    df_strat_smoothed, 'stable_asymptomatic', 'stable_symptomatic', d1, smoothing_length=length, method='weighted')

In [ ]:
d1 = '2020-06-15'
length = (datetime.datetime.strptime(d1, '%Y-%m-%d') - df_strat_smoothed.loc[0, 'date']).days
df_strat_smoothed = smooth_big_jump_helper(
    df_strat_smoothed, 'ventilator', 'icu', d1, smoothing_length=length, method='weighted-mag')

In [ ]:
d1 = '2020-06-15'
length = (datetime.datetime.strptime(d1, '%Y-%m-%d') - df_strat_smoothed.loc[0, 'date']).days
df_strat_smoothed = smooth_big_jump_helper(
    df_strat_smoothed, 'icu', 'o2_beds', d1, smoothing_length=length, method='weighted-mag')

d1 = '2020-05-31'
length = (datetime.datetime.strptime(d1, '%Y-%m-%d') - df_strat_smoothed.loc[0, 'date']).days
df_strat_smoothed = smooth_big_jump_helper(
    df_strat_smoothed, 'icu', 'o2_beds', d1, smoothing_length=length, method='weighted-mag')

In [ ]:
d1 = '2020-06-08'
length = (datetime.datetime.strptime(d1, '%Y-%m-%d') - df_strat_smoothed.loc[0, 'date']).days
df_strat_smoothed = smooth_big_jump_helper(
    df_strat_smoothed, 'o2_beds', 'non_o2_beds', d1, smoothing_length=length, method='weighted-mag')

d1 = '2020-06-14'
length = (datetime.datetime.strptime(d1, '%Y-%m-%d') - df_strat_smoothed.loc[0, 'date']).days
df_strat_smoothed = smooth_big_jump_helper(
    df_strat_smoothed, 'o2_beds', 'non_o2_beds', d1, smoothing_length=length, method='weighted-mag')

d1 = '2020-06-15'
length = (datetime.datetime.strptime(d1, '%Y-%m-%d') - df_strat_smoothed.loc[0, 'date']).days
df_strat_smoothed = smooth_big_jump_helper(
    df_strat_smoothed, 'o2_beds', 'non_o2_beds', d1, smoothing_length=length, method='weighted-mag')

In [ ]:
d1 = '2020-05-28'
length = (datetime.datetime.strptime(d1, '%Y-%m-%d') - df_strat_smoothed.loc[0, 'date']).days
df_strat_smoothed = smooth_big_jump_helper(
    df_strat_smoothed, 'non_o2_beds', 'hq', d1, smoothing_length=length, method='weighted-mag')

d1 = '2020-05-31'
length = (datetime.datetime.strptime(d1, '%Y-%m-%d') - df_strat_smoothed.loc[0, 'date']).days
df_strat_smoothed = smooth_big_jump_helper(
    df_strat_smoothed, 'non_o2_beds', 'hq', d1, smoothing_length=length, method='weighted-mag')

In [ ]:
d1 = '2020-06-14'
length = (datetime.datetime.strptime(d1, '%Y-%m-%d') - df_strat_smoothed.loc[0, 'date']).days
df_strat_smoothed = smooth_big_jump_helper(
    df_strat_smoothed, 'non_o2_beds', 'o2_beds', d1, smoothing_length=length, method='weighted-mag')

d1 = '2020-06-19'
length = (datetime.datetime.strptime(d1, '%Y-%m-%d') - df_strat_smoothed.loc[0, 'date']).days
df_strat_smoothed = smooth_big_jump_helper(
    df_strat_smoothed, 'non_o2_beds', 'o2_beds', d1, smoothing_length=length, method='weighted-mag')

In [ ]:
d1 = '2020-06-09'
length = (datetime.datetime.strptime(d1, '%Y-%m-%d') - df_strat_smoothed.loc[0, 'date']).days
df_strat_smoothed = smooth_big_jump_helper(
    df_strat_smoothed, 'hq', 'non_o2_beds', d1, smoothing_length=length, method='weighted-mag')

d1 = '2020-06-17'
length = (datetime.datetime.strptime(d1, '%Y-%m-%d') - df_strat_smoothed.loc[0, 'date']).days
df_strat_smoothed = smooth_big_jump_helper(
    df_strat_smoothed, 'hq', 'non_o2_beds', d1, smoothing_length=length, method='weighted-mag')

In [ ]:
d1 = '2020-06-17'
length = (datetime.datetime.strptime(d1, '%Y-%m-%d') - df_strat_smoothed.loc[0, 'date']).days
df_strat_smoothed = smooth_big_jump_helper(
    df_strat_smoothed, 'non_o2_beds', 'hq', d1, smoothing_length=length, method='weighted-mag')

In [ ]:
d1 = '2020-06-14'
length = (datetime.datetime.strptime(d1, '%Y-%m-%d') - df_strat_smoothed.loc[0, 'date']).days
df_strat_smoothed = smooth_big_jump_helper(
    df_strat_smoothed, 'o2_beds', 'non_o2_beds', d1, smoothing_length=length, method='weighted-mag')

In [ ]:
d1 = '2020-06-14'
length = (datetime.datetime.strptime(d1, '%Y-%m-%d') - df_strat_smoothed.loc[0, 'date']).days
df_strat_smoothed = smooth_big_jump_helper(
    df_strat_smoothed, 'non_o2_beds', 'hq', d1, smoothing_length=length, method='weighted-mag')

In [ ]:
d1 = '2020-06-15'
length = (datetime.datetime.strptime(d1, '%Y-%m-%d') - df_strat_smoothed.loc[0, 'date']).days
df_strat_smoothed = smooth_big_jump_helper(
    df_strat_smoothed, 'non_o2_beds', 'hq', d1, smoothing_length=length, method='weighted-mag')

d1 = '2020-06-09'
length = (datetime.datetime.strptime(d1, '%Y-%m-%d') - df_strat_smoothed.loc[0, 'date']).days
df_strat_smoothed = smooth_big_jump_helper(
    df_strat_smoothed, 'non_o2_beds', 'hq', d1, smoothing_length=length, method='weighted-mag')

In [ ]:
plot_smoothing(df_strat, df_strat_smoothed, 'Maharashtra', 'Mumbai', which_compartments=all_compartments, description='Smoothing')